In [17]:
import time
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

In [89]:
def create_dataset(index_start, index_finish):
    """GET DATA FROM MATCH 11x11.ru/reports/...
    
    PARAMETERS:
    -----------------------
    int index_start :: start parse with 11x11.ru/reports/start
    int index_finish :: start parse with 11x11.ru/reports/finish
    
    RETURN:
    -----------------------
    list of dictionaries :: [{data_start}, {data_two}, ..., {data_finish}],
    data is time match, name players, viewers match, str. on field, score match, type blows, stratety, 
    see more on example.png on dir.
    """
    
    # Autohorization
    data = {
        'auth_name':'webscrap',
        'auth_pass':'webscrap1'
    }
    session = requests.session()
    session.post('http://11x11.ru/', data=data)
    
    ## Add data on list
    result_data = []
    errors = 0
    for i in range(index_start, index_finish):
        try: 
            data = BeautifulSoup(session.get("http://11x11.ru/reports/" + str(i)).text)

            data_table = data('td', colspan = 2)[1]('td', align = None)[2:]
            data_table_right = data('td', colspan = 2)[1]('td', align = 'right')[2:]
            
            time = data('b')[0].string
            viewers = data('p')[-1].string
            str_on_field, disposal, tactic, pressing, strategy, blows = map(lambda x: x.string, data_table)
            str_on_field_r, disposal_r, tactic_r, pressing_r, strategy_r, blows_r = map(lambda x: x.string, data_table_right)
            p1_name = data('h3')[2]('a')[0].string
            p2_name = data('h3')[3]('a')[0].string
            score = data('font')
            result_data.append({
                'time' : time,
                'viewers' : viewers,
                'str_on_field_1' : str_on_field,
                'str_on_field_2' : str_on_field_r,
                'disposal_1' : disposal,
                'disposal_2' : disposal_r,
                'tactic_1' : tactic,
                'tactic_2' : tactic_r,
                'pressing_1' : pressing,
                'pressing_2' : pressing_r, 
                'strategy_1' : strategy,
                'strategy_2' : strategy_r,
                'player_1' : p1_name,
                'player_2' : p2_name,
                'blows_1' : blows,
                'blows_2' : blows_r,
                'id_match' : i,
                'score_player_1' : score[0].string.split(':')[0],
                'score_player_2' : score[0].string.split(':')[1]
            })
        except:
            errors += 1
    return {'data' : result_data, 'errors' : errors}

def find_last_match_id():
    """Find last match id in real time"""
    try:
        data = {
        'auth_name':'webscrap',
        'auth_pass':'webscrap1'
        }
        session = requests.session()
        session.post('http://11x11.ru/', data=data)
    except:
        return 'Auhtorization failed'
    try:
        return int((BeautifulSoup(session.get('http://11x11.ru/xml/games/history.php?act=fullhistory').text).
               findAll(href = re.compile('/reports'))[0]['href']).split('/')[-1])
    except:
        return "Not find last id match"
def dowland_chunks(numbers_chunks=3, lenght_cnunks=1000):
    """Dowlands parts datasets
    
    PARAMETERS:
    --------------------------
    int numbers_chunks :: numbers chunks for dowlands
    int lenght_chunks :: lenght chunks
    
    RETURN:
    --------------------------
    list datasets :: list of datasets (in the series form)
    """
    
    datasets = []
    last_id = find_last_match_id()
    for i in reversed(range(numbers_chunks)):
        print(last_id - lenght_cnunks * (i + 1) + 1, last_id - lenght_cnunks * i)
        tmp_df = pd.DataFrame(create_dataset(last_id - lenght_cnunks * (i + 1) + 1, 
                                                 last_id - lenght_cnunks * i))['data']
        datasets.append(tmp_df)

    return datasets

def chucks_to_dataframe(chunks):
    tempory_df = pd.DataFrame()
    for df in chunks:
        tempory_df = pd.concat([tempory_df, pd.DataFrame(map(lambda x: dict(x), df))])
    return tempory_df


In [91]:
tempory = chucks_to_dataframe(dowland_chunks(numbers_chunks=10, lenght_cnunks=100))

951714784 951714883
951714884 951714983
951714984 951715083
951715084 951715183
951715184 951715283
951715284 951715383
951715384 951715483
951715484 951715583
951715584 951715683
951715684 951715783


In [96]:
tempory

,time,viewers,str_on_field_1,str_on_field_2,disposal_1,disposal_2,tactic_1,tactic_2,pressing_1,pressing_2,strategy_1,strategy_2,player_1,player_2,blows_1,blows_2,id_match,score_player_1,score_player_2
0,27.12.2019 21:36,Зрители: 17835,153.7,58.2,4-3-3,4-4-2,защитная (27%),защитная (38%),нет,нет,нормальная,нормальная,Танкоград,Ava Kursk,дальние,дальние,951714784,4,1
1,27.12.2019 21:35,Зрители: 0,91.9,165.1,4-4-2,4-3-3,нормальная (50%),нормальная (47%),None,None,нормальная,игра в пас,Спарринг-Бот,кивиыли,смешанные,дальние,951714785,0,6
2,27.12.2019 21:37,Зрители: 41803,164.2,91.9,3-4-3,4-4-2,суперзащитная (10%),нормальная (50%),None,None,дальние удары,нормальная,ibiza163,Спарринг-Бот,дальние,смешанные,951714786,3,0
3,27.12.2019 21:37,Зрители: 51,109.2,91.9,3-5-2,4-4-2,нормальная (50%),нормальная (50%),None,None,техничная игра,нормальная,Belkanick,Спарринг-Бот,смешанные,смешанные,951714788,4,2
4,27.12.2019 21:39,Зрители: 0,91.9,149.1,4-4-2,3-4-3,нормальная (50%),суперзащитная (11%),None,None,нормальная,дальние удары,Спарринг-Бот,Tyumen-City,смешанные,дальние,951714789,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,27.12.2019 21:41,None,201,168.6,3-4-3,2-6-2,суперзащитная (19%),суперзащитная (1%),нет,нет,дальние удары,дальние удары,Cristall,lik13,дальние,дальние,951715775,1,0
70,27.12.2019 21:43,None,190.5,135.8,2-6-2,4-3-3,защитная (38%),суперзащитная (2%),нет,нет,техничная игра,дальние удары,султике,kingkong,дальние,дальние,951715778,0,4
71,27.12.2019 21:40,Зрители: 0,191.5,124.8,4-4-2,4-4-2,нормальная (50%),нормальная (47%),None,None,нормальная,нормальная,Спарринг-Бот,Blondинка,смешанные,дальние,951715779,0,6
72,27.12.2019 21:43,Зрители: 0,191.5,112.6,4-4-2,3-5-2,нормальная (50%),суперзащитная (8%),None,None,нормальная,дальние удары,Спарринг-Бот,lliissiiyy,смешанные,дальние,951715780,0,2
